In [21]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import datetime
import time
import smtplib
import os

In [31]:
# Connect and pull data from Amazon
URL = 'https://www.amazon.de/Steve-Jobs-Walter-Isaacson/dp/1451648537'
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
page = requests.get(URL, headers=HEADERS)
soup1 = BeautifulSoup(page.content, "html.parser")

In [23]:
# Check and clean up data
title = soup1.find(id='productTitle').get_text().strip()
title

'Steve Jobs: A Biography'

In [24]:
# Check and clean up data
price = soup1.find(class_='a-size-base a-color-price a-color-price').get_text().strip()[0:5]
price = price.replace(',', '.') 
price = float(price)
price

25.99

In [25]:
# Create a Timestamp, to track the collected data
dtime = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
dtime

'15/11/2022 17:02:19'

In [26]:
# Create a CSV file and appending data to it
header = ['Title', 'Price', 'Datetime']
data = [title, price, dtime]

with open('Web_Scraping_Project_Amazon_Dataset.csv', 'a+', newline = '', encoding='UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [27]:
# Check the CSV File
df = pd.read_csv('Web_Scraping_Project_Amazon_Dataset.csv')
df

,Title,Price,Datetime
0,Steve Jobs: A Biography,25.99,15/11/2022 17:02:19


In [28]:
# Create a function with the steps above for iteration purposes
def auto_csv():
    URL = 'https://www.amazon.de/Steve-Jobs-Walter-Isaacson/dp/1451648537'
    HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    
    page = requests.get(URL, headers=HEADERS)
    
    soup1 = BeautifulSoup(page.content, "html.parser")
    
    title = soup1.find(id='productTitle').get_text().strip()
    
    price = soup1.find(class_='a-size-base a-color-price a-color-price').get_text().strip()[0:5]
    
    dtime = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    
    header = ['Title', 'Price', 'Datetime']
    data = [title, price, dtime]

    with open('Web_Scraping_Project_Amazon_Dataset.csv', 'a+', newline = '', encoding='UTF-8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

In [29]:
# Create a function for sending an email when desired events occur
def auto_mail():
    gmail_user = os.environ.get('GMAIL_USER')
    gmail_password = os.environ.get('GMAIL_PASSWORD')

    sent_from = gmail_user
    to = [os.environ.get('RECIPIENTS_EMAIL')]
    subject = 'Purchase Note'
    body = f'{title} is available at a low price of {price}: \n{URL}'
    email_text = """\
    From: %s
    To: %s
    Subject: %s

    %s
    """ % (sent_from, ", ".join(to), subject, body)

    try:
        smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        smtp_server.ehlo()
        smtp_server.login(gmail_user, gmail_password)
        smtp_server.sendmail(sent_from, to, email_text)
        smtp_server.close()
        print ("Email sent successfully!")
    except Exception as ex:
        print ("Something went wrong!",ex)

In [30]:
# Repeating the code every hour
while(True):
    auto_csv()
    
    # Set a condition for sending emails
    if price < 30.00:
        auto_mail()
    
    time.sleep(5)

Email sent successfully!
Email sent successfully!
Email sent successfully!
Email sent successfully!
Email sent successfully!


KeyboardInterrupt: 